# 过滤

## 需要过滤的原因

因为：

- 交易所会在非交易时间发送 tick 数据，大多是结算数据（最终持仓量）；
- pandas resample 会生成非交易时间数据。

所以需要丢弃非交易时间的数据。

## 判断交易时间

判断交易时间：

    开盘时间 <= t <= 收盘时间
    
这可以使用 pandas 的 bool index （布尔索引）来快速实现，以上期所的螺纹钢2105为例：

In [1]:
from pathlib import Path
import datetime as dt

import pandas as pd

from src.utility import PACKAGE_PATH


# 定义数据文件
data_file: str = 'SHFE.rb2105_Tick.csv'

# 转为Path格式。
data_path: Path = PACKAGE_PATH.joinpath('data', data_file)

# 加载tick数据为DataFrame。
# 【parse_dates=['datetime']】表示在加载时将 datetime 字段解读为 datetime64 格式。
# 【index_col=['datetime']】表示在加载时将 datetime 字段作为 DateTimeIndex。
df: pd.DataFrame = pd.read_csv(data_path, parse_dates=['datetime'], index_col=['datetime'])

# 显示 df。
print('未过滤的数据:\n', df)

未过滤的数据:
                                   datetime_nano  SHFE.rb2105.last_price  \
datetime                                                                  
2020-05-15 19:43:23.300000  1589543003300000000                     NaN   
2020-05-15 20:59:00.500000  1589547540500000000                  3230.0   
2020-05-15 21:00:00.500001  1589547600500001000                  3231.0   
2020-05-15 21:00:01.000000  1589547601000000000                  3235.0   
2020-05-15 21:00:01.500000  1589547601500000000                  3230.0   
...                                         ...                     ...   
2021-05-17 14:24:49.500000  1621232689500000000                     NaN   
2021-05-17 14:38:03.500000  1621233483500000000                     NaN   
2021-05-17 14:59:55.000000  1621234795000000000                     NaN   
2021-05-17 14:59:59.000001  1621234799000001000                     NaN   
2021-05-17 14:59:59.999500  1621234799999500000                  5580.0   

               

In [2]:
# 过滤 df。
df2 = df[
    (
        ((df.index.time >= dt.datetime.strptime('21:00', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('23:00', '%H:%M').time())) |
        ((df.index.time >= dt.datetime.strptime('09:00', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('10:15', '%H:%M').time())) |
        ((df.index.time >= dt.datetime.strptime('10:30', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('11:30', '%H:%M').time())) |
        ((df.index.time >= dt.datetime.strptime('13:30', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('15:00', '%H:%M').time()))
    )
]

# 显示 df。
print('过滤后的数据:\n', df2)

过滤后的数据:
                                   datetime_nano  SHFE.rb2105.last_price  \
datetime                                                                  
2020-05-15 21:00:00.500001  1589547600500001000                  3231.0   
2020-05-15 21:00:01.000000  1589547601000000000                  3235.0   
2020-05-15 21:00:01.500000  1589547601500000000                  3230.0   
2020-05-15 21:00:02.000000  1589547602000000000                  3230.0   
2020-05-15 21:00:02.500000  1589547602500000000                  3222.0   
...                                         ...                     ...   
2021-05-17 14:24:49.500000  1621232689500000000                     NaN   
2021-05-17 14:38:03.500000  1621233483500000000                     NaN   
2021-05-17 14:59:55.000000  1621234795000000000                     NaN   
2021-05-17 14:59:59.000001  1621234799000001000                     NaN   
2021-05-17 14:59:59.999500  1621234799999500000                  5580.0   

               

In [3]:
print(f'过滤掉 {len(df) - len(df2)} 行数据。')

过滤掉 484 行数据。


## 问题

注意布尔索引的第一行：

`
  ((df.index.time >= dt.datetime.strptime('21:00', '%H:%M').time()) & (df.index.time <= dt.datetime.strptime('23:00', '%H:%M').time()))
`

这是螺纹钢夜盘的时间，从晚上九点到晚上十一点。大部分夜盘品种也是这个时间。

然而贵金属、原油以及有色金属不是这样的，他们在晚上九点开始，但是收盘时间要么是第二天的凌晨两点半（贵金属和原油），要么是凌晨一点（有色），都跨过了子夜。

这就不好使用一个逻辑表达式来过滤了。

简单的办法，将北京时间转化一下，在一个结算日内，交易时间都是连续递增的。

比如北京时间的晚上八点，作为一个新的时区的 00:00（这其实就是新西兰时间）。